In [2]:
# Core scientific and visualization libraries
!pip install numpy matplotlib seaborn scikit-learn

# Keras and TensorFlow (Keras is now part of TensorFlow)
!pip install tensorflow

# If using standalone Keras (not recommended in new projects):
# pip install keras

# Image preprocessing utilities
!pip install pillow


Defaulting to user installation because normal site-packages is not writeable
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl (11.1 MB)
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   - -------------------------------------- 10.2/307.7 kB ? eta -:--:--
   --- ----------------------------------- 30.7/307.7 kB 660.6 kB/s eta 0:00:01
   ------- ------------------------------- 61.4/307.7 kB 656.4 kB/s eta 0:00:01
   ---------------------- ----------------- 174.1/307.7 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 307.7/307.7 kB 1.6 MB/s eta 0:00:00
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.17.2-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached h5py-3.13.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib.pyplot import imshow
from keras.models import load_model
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import KFold
import seaborn as sns

ImportError: Traceback (most recent call last):
  File "d:\code\.venv\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [33]:
#root = '101_ObjectCategories'
#exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
root = 'D:/Milon2/final_CNN/Chunk_average_spectro'
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
#categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]

print(categories)

['D:/Milon2/final_CNN/Chunk_average_spectro\\HealthyControl', 'D:/Milon2/final_CNN/Chunk_average_spectro\\Schizophrenia']


In [34]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [35]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

In [36]:
num_folds = 10
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_no = 1

for train, test in kfold.split(data):
    # Create output file paths using f-strings
    train_path = f"D:/Milon2/final_CNN/result/data/train/train{fold_no}.csv"
    test_path = f"D:/Milon2/final_CNN/result/data/test/test{fold_no}.csv"

    # Save if not already present
    if not os.path.exists(train_path):
        np.savetxt(train_path, train, delimiter=",", fmt='%s')
        np.savetxt(test_path, test, delimiter=",", fmt='%s')
    else:
        print("exists")

    fold_no += 1

In [37]:
X, y = np.array([t["x"] for t in data]), np.array([t["y"] for t in data])

# normalize data
X = X.astype('float32') / 255.



In [38]:
# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []

import tensorflow as tf
import pandas as pd
from tensorflow.keras import utils as np_utils

tf.keras.backend.clear_session()

# K-fold Cross Validation model evaluation
fold_no = 1
for fold_no in range(1, 11):
    
    train=pd.read_csv(f"D:/Milon2/final_CNN/result/data/train/train{fold_no}.csv", sep=',',header=None)
    test=pd.read_csv(f"D:/Milon2/final_CNN/result/data/test/test{fold_no}.csv", sep=',',header=None)
    
    x_train = X[train[0]]
    y_train = y[train[0]]
    x_test = X[test[0]]
    y_test = y[test[0]]

    # x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
    # x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]

    # normalize data
    # x_train = x_train.astype('float32') / 255.
    # x_test = x_test.astype('float32') / 255.

    # convert labels to one-hot vectors
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)
    print(y_test.shape)

    # summary
    print("finished loading %d images from %d categories"%(len(data), num_classes))
    print("train / test split: %d, %d"%(len(x_train), len(x_test)))
    print("training data shape: ", x_train.shape)
    print("training labels shape: ", y_train.shape)

    #x_train = X[train]
    #y_train = y[train]
    #x_test = X[test]
    #y_test = y[test]
    
    
    # build the network
    model = Sequential()
    #print("Input dimensions: ",x_train.shape[1:])

    model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))

    model.add(Dropout(0.5))

    model.add(Dense(2))
    model.add(Activation('softmax'))


    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.summary()
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    #filepath="C:/Users/s4629860/Documents/Workspace/SZ/Dataset 5/CNN_epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
    filepath=f"D:/Milon2/final_CNN/result/model/best_model{fold_no}.best.keras"
    
    # define early stopping callback
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=150, mode='auto')

    # saves the model weights after each epoch if the validation loss decreased
    #checkpointer = ModelCheckpoint(filepath='LSTM_model.best.hdf5', verbose=1, save_best_only=True)
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True, monitor='val_accuracy')

    callbacks_list = [earlystop, checkpointer]
    
    history = model.fit(x_train, y_train, epochs = 20, batch_size=32,validation_data= (x_test, y_test), callbacks = callbacks_list)

    #load saved best model
    model = load_model(filepath)

    # Generate generalization metrics
    scores = model.evaluate(x_test, y_test, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    Y_pred = model.predict(x_test)
    y_pred = np.argmax(Y_pred, axis=1)
    y_text_new = np.argmax(y_test, axis=1)

    from sklearn.metrics import classification_report, confusion_matrix
    #Confution Matrix and Classification Report

    print('Confusion Matrix')
    confusion = confusion_matrix(y_text_new, y_pred)
    print(confusion)
    print('Classification Report')
    target_names = ['AD', 'Normal']
    print(classification_report(y_text_new, y_pred, target_names=target_names))

    fig = plt.figure(figsize=(16,4))
    ax = fig.add_subplot(121)
    ax.plot(history.history["val_loss"])
    ax.set_title("validation loss")
    ax.set_xlabel("epochs")

    ax2 = fig.add_subplot(122)
    ax2.plot(history.history["val_accuracy"])
    ax2.set_title("validation accuracy")
    ax2.set_xlabel("epochs")
    ax2.set_ylim(0, 1)

    # Increase fold number
    #fold_no = fold_no + 1
    plt.show()
    
    ax= plt.subplot()
    sns.heatmap(confusion, annot=True, fmt="d", ax = ax); #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['ScZ', 'Healthy']); ax.yaxis.set_ticklabels(['ScZ', 'Healthy']);
    plt.show()    

(939, 2)
finished loading 9381 images from 2 categories
train / test split: 8442, 939
training data shape:  (8442, 224, 224, 3)
training labels shape:  (8442, 2)


C:\Users\Milon\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 222, 222, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 109, 109, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 52, 52, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,209,058 (4.61 MB)

 Trainable params: 1,209,058 (4.61 MB)

 Non-trainable params: 0 (0.00 B)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.6031 - loss: 0.6378
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to D:/Milon2/final_CNN/result/model/best_model1.best.keras
264/264 ━━━━━━━━━━━━━━━━━━━━ 56s 202ms/step - accuracy: 0.6036 - loss: 0.6370 - val_accuracy: 1.0000 - val_loss: 9.5956e-04
Epoch 2/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.9989 - loss: 0.0048
Epoch 2: val_accuracy did not improve from 1.00000
264/264 ━━━━━━━━━━━━━━━━━━━━ 53s 201ms/step - accuracy: 0.9989 - loss: 0.0048 - val_accuracy: 1.0000 - val_loss: 3.8809e-06
Epoch 3/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.9998 - loss: 6.5661e-04
Epoch 3: val_accuracy did not improve from 1.00000
264/264 ━━━━━━━━━━━━━━━━━━━━ 53s 200ms/step - accuracy: 0.9998 - loss: 6.5800e-04 - val_accuracy: 1.0000 - val_loss: 3.6408e-06
Epoch 4/20
264/264 ━━━━━━━━━━━━━

KeyboardInterrupt: 

In [32]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 1.7125767044490203e-05 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.00016609788872301579 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.0005249115056358278 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.0003415822866372764 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 5 - Loss: 1.988848998735193e-05 - Accuracy: 100.0%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 100.0 (+- 0.0)
> Loss: 0.0002139211876055924
------------------------------------------------------------------------
